In [7]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

In [1]:
import gpxpy
import pandas as pd

In [4]:
def gpx_to_dataframe(gpx_file_path):
    with open(gpx_file_path, 'r', encoding='UTF8') as gpx_file:
        gpx = gpxpy.parse(gpx_file)

    data = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                data.append({
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'elevation': point.elevation,
                    'time': point.time,
                })

    df = pd.DataFrame(data)
    return df

In [5]:
gpx_file_path = "./우면산-양재시민의숲-매헌윤봉길의사기념관-구룡산-대모산-인릉산_20200119_104402(aecdre@hanmail.net-20200119_164430).gpx"
gpx_dataframe = gpx_to_dataframe(gpx_file_path)

gpx_dataframe

,latitude,longitude,elevation,time
0,37.475599,126.981943,54.319719,2020-01-19 10:44:02+00:00
1,37.475504,126.981965,52.937993,2020-01-19 10:44:20+00:00
2,37.475403,126.981997,51.403960,2020-01-19 10:44:33+00:00
3,37.475300,126.982010,51.777725,2020-01-19 10:44:42+00:00
4,37.475195,126.982029,51.225949,2020-01-19 10:44:57+00:00
...,...,...,...,...
1825,37.431126,127.072073,127.816744,2020-01-19 16:43:32+00:00
1826,37.431019,127.072016,127.985176,2020-01-19 16:43:39+00:00
1827,37.430935,127.072069,127.366314,2020-01-19 16:43:47+00:00
1828,37.430855,127.072125,125.711641,2020-01-19 16:43:55+00:00


In [33]:
from tqdm import tqdm 

In [42]:
import folium
from folium.plugins import TimestampedGeoJson
import pandas as pd

# gpx_dataframe: latitude, longitude, elevation, time 정보가 담긴 시계열 트랙 데이터프레임

# Create a base map centered at the mean latitude and longitude
center_lat = gpx_dataframe['latitude'].mean()
center_lon = gpx_dataframe['longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Create a GeoJSON FeatureCollection with properties including time
feature_collection = {
    "type": "FeatureCollection",
    "features": []
}

# Define a function to determine marker color based on elevation
def get_marker_color(elevation):
    if elevation < 50:
        return 'green'
    elif elevation < 100:
        return 'orange'
    else:
        return 'red'

# Add custom markers for each data point
for index, row in gpx_dataframe.iterrows():
    marker = folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color=get_marker_color(row['elevation']),
        fill=True,
    )
    marker.add_to(m)

for index, row in tqdm(gpx_dataframe.iterrows()):
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row['longitude'], row['latitude']],
        },
        "properties": {
            "time": row['time'].isoformat(),
            "elevation": row['elevation'],
        },
    }
    feature_collection['features'].append(feature)

# Add TimestampedGeoJson plugin to the map with adjusted options
TimestampedGeoJson(
    feature_collection,
    period="PT5S",  # Adjust the time interval as needed
    add_last_point=True,
    auto_play=True,
    loop=True,
    max_speed=1,  # Adjust the speed (lower value is slower)
    date_options="YYYY-MM-DD HH:mm:ss",  # Adjust the date display format
).add_to(m)  # Add the point_to_layer function directly
    

# Save the map to an HTML file
m.save("custom_marker_track_map.html")

1830it [00:00, 13615.33it/s]
